In [1]:
# !pip install h5py
# !pip install pandas matplotlib
# !pip install seaborn
# !pip install scipy
# !pip install scikit-learn

In [2]:
import torch
import pandas
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from matplotlib.patches import Patch
import copy
import h5py
import itertools
import os

from model.model_3D import *
from model.train_eval import *

from torch.utils.data import DataLoader, TensorDataset
from torch.optim.lr_scheduler import StepLR

from postprocessing.save_results import *
from postprocessing.plot_results import *
from postprocessing.metrics import single_roc_curve

In [3]:
### check if cuda is available

if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

print(f"Using {device}.")

Using cuda.


In [4]:
### import NDVI data

def load_with_hdf5(name):
    """
    Load tensor data from HDF5 with gzip compression
    """
    with h5py.File(f"data/{name}.h5", "r") as f:
        loaded_features = torch.from_numpy(f["features"][:])
        loaded_labels = torch.from_numpy(f["labels"][:])
    return TensorDataset(loaded_features, loaded_labels)

train_set = load_with_hdf5("train_set_ndvi_v7")
val_set = load_with_hdf5("val_set_ndvi_v7")
test_set = load_with_hdf5("test_set_ndvi_v7")


In [5]:
print(f"Train set: {len(train_set)} samples")
print(f"Validation set: {len(val_set)} samples")
print(f"Test set: {len(test_set)} samples")

Train set: 518 samples
Validation set: 19 samples
Test set: 19 samples


In [6]:
print(train_set.tensors[0].shape)
print(train_set.tensors[1].shape)

torch.Size([518, 2, 4, 1000, 500])
torch.Size([518, 1000, 500])


In [7]:
def train_and_validate(batch_size, learning_rate, init_hid_dim, num_epochs=100, device='cuda'):
    # Initialize model
    model = UNet3D(
        n_channels=train_set[0][0].shape[0],
        n_classes=1,
        init_hid_dim=init_hid_dim,
        kernel_size=3,
        pooling='max',
        bilinear=False,
        drop_channels=False
    )

    torch.cuda.empty_cache()

    num_parameters = sum(p.numel() for p in model.parameters())
    print(f"Number of parameters: {num_parameters:.2e}.")

    optimizer = torch.optim.Adam(params=model.parameters(), lr=learning_rate)
    scheduler = StepLR(optimizer, step_size=15, gamma=0.75)

    # Data loaders
    train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_set, batch_size=batch_size, shuffle=False)

    best_loss = float('inf')
    best_model = None

    train_losses = []
    val_losses = []
    accuracies = []
    precisions = []
    recalls = []
    f1_scores = []
    csi_scores = []

    # Training loop
    for epoch in range(1, num_epochs + 1):
        # Training step
        train_loss = training_unet(model, train_loader, optimizer, device=device, loss_f='BCE', water_threshold=0.5)
        train_losses.append(train_loss)

        # Validation step
        val_loss, val_accuracy, val_precision, val_recall, val_f1_score, val_csi_score = validation_unet(
            model, val_loader, device=device, loss_f='BCE', water_threshold=0.5
        )
        val_losses.append(val_loss)
        accuracies.append(val_accuracy)
        precisions.append(val_precision)
        recalls.append(val_recall)
        f1_scores.append(val_f1_score)
        csi_scores.append(val_csi_score)

        # Update learning rate
        scheduler.step()

        # Save best model
        if val_loss < best_loss:
            best_loss = val_loss
            best_model = copy.deepcopy(model)

        # Print progress every 20 epochs
        if epoch % 1 == 0:
            print(f"Epoch: {epoch} | Training Loss: {train_loss:.4f}, Validation Loss: {val_loss:.4f}, "
                  f"Best Validation Loss: {best_loss:.4f}")
            print(f"Metrics | Accuracy: {val_accuracy:.3f}, Precision: {val_precision:.3f}, Recall: {val_recall:.3f}, "
                  f"F1-score: {val_f1_score:.3f}, CSI-score: {val_csi_score:.3f}")
            print(f"Learning Rate: {scheduler.get_last_lr()[0]:.6f}")

    return best_loss, best_model, train_losses, val_losses, accuracies, precisions, recalls, f1_scores, csi_scores

In [11]:
# Test first with Antonio's Hyperparameters
batch_size = 16
learning_rate = 0.05
init_hid_dim = 32
num_epochs = 1
device = 'cuda' if torch.cuda.is_available() else 'cpu'
loss_f = 'BCE'
machine = 'machine_1'

# Train and validate
print(f"Running training for Batch Size={batch_size}, Learning Rate={learning_rate}, Init Hid Dim={init_hid_dim}")
val_loss, best_model, train_losses, val_losses, accuracies, precisions, recalls, f1_scores, csi_scores = train_and_validate(
    batch_size=batch_size,
    learning_rate=learning_rate,
    init_hid_dim=init_hid_dim,
    num_epochs=num_epochs,
    device=device
)

# Save the best model
save_model_path(
    machine=machine,
    model=best_model,
    batch_size=batch_size,
    learning_rate=learning_rate,
    init_hid_dim=init_hid_dim,
    epochs=num_epochs,
    dir_output="model/models_trained"
)

# Save training and validation metrics
save_losses_metrics(
    machine=machine,
    train_losses=train_losses,
    val_losses=val_losses,
    metrics=[accuracies, precisions, recalls, f1_scores, csi_scores],
    batch_size=batch_size, 
    learning_rate=learning_rate, 
    init_hid_dim=init_hid_dim, 
    epochs=num_epochs,
    dir_output="model/losses_metrics" 
)

# Define the test loader
test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=False)

# Test the best model
model_loss = copy.deepcopy(best_model)
test_loss, test_accuracy, test_precision, test_recall, test_f1_score, test_csi_score = validation_unet(
    model_loss, test_loader, device=device, loss_f=loss_f
)

print(f'Average metrics for test dataset using model with best validation loss:\n\n\
BCE loss:          {test_loss:.3e}\n\
Accuracy:          {test_accuracy:.3f}\n\
Precision:         {test_precision:.3f}\n\
Recall:            {test_recall:.3f}\n\
F1 score:          {test_f1_score:.3f}\n\
CSI score:         {test_csi_score:.3f}')

Running training for Batch Size=16, Learning Rate=0.05, Init Hid Dim=32
Number of parameters: 7.77e+06.
Epoch: 1 | Training Loss: 0.2500, Validation Loss: 2.8002, Best Validation Loss: 2.8002
Metrics | Accuracy: 0.898, Precision: 0.516, Recall: 0.349, F1-score: 0.416, CSI-score: 0.263
Learning Rate: 0.050000
Metrics saved at: model/losses_metrics/losses_metrics_NDVI_bs16_lr0.05_hid32_epoch1.csv
Average metrics for test dataset using model with best validation loss:

BCE loss:          3.124e+00
Accuracy:          0.901
Precision:         0.608
Recall:            0.395
F1 score:          0.479
CSI score:         0.315


In [10]:
# Hyperparameter Optimization

# Define hyperparameter values
batch_sizes = [8, 16, 32]
learning_rates = [0.01, 0.05, 0.1]
init_hid_dims = [8, 16, 32]
num_epochs = 1
loss_f = 'BCE'  # Define loss function for training and testing
machine='machine_1'

# Split combinations across machines
all_combinations = list(itertools.product(batch_sizes, learning_rates, init_hid_dims))
split_index = len(all_combinations) // 2
combinations_machine_1 = all_combinations[:split_index]
combinations_machine_2 = all_combinations[split_index:]

# Select the current machine's combinations
combinations = combinations_machine_2  # Change to combinations_machine_1 on Machine 1

# Results storage
results = []

# Run grid search
for batch_size, learning_rate, init_hid_dim in combinations:
    print(f"Testing: Batch size={batch_size}, Learning rate={learning_rate}, Init Hid Dim={init_hid_dim}")
    
    # Train and validate
    val_loss, best_model, train_losses, val_losses, accuracies, precisions, recalls, f1_scores, csi_scores = train_and_validate(
        batch_size=batch_size,
        learning_rate=learning_rate,
        init_hid_dim=init_hid_dim,
        num_epochs=num_epochs,
        device=device
    )
    
    # Save the best model
    save_model_path(
        machine=machine,
        model=best_model,
        batch_size=batch_size,
        learning_rate=learning_rate,
        init_hid_dim=init_hid_dim,
        epochs=num_epochs,
        dir_output="model/models_trained"
    )
    
    # Save training and validation metrics
    save_losses_metrics(
        machine=machine,
        train_losses=train_losses,
        val_losses=val_losses,
        metrics=[accuracies, precisions, recalls, f1_scores, csi_scores],
        batch_size=batch_size, 
        learning_rate=learning_rate, 
        init_hid_dim=init_hid_dim, 
        epochs=num_epochs,
        dir_output="model/losses_metrics" 
    )

    # Define the test loader
    test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=False)
    
    # Test the best model on the test dataset
    model_loss = copy.deepcopy(best_model)
    test_loss, test_accuracy, test_precision, test_recall, test_f1_score, test_csi_score = validation_unet(
        model_loss, test_loader, device=device, loss_f=loss_f
    )
    
    print(f'Average metrics for test dataset using model with best validation loss:\n\n\
{loss_f} loss:          {test_loss:.3e}\n\
Accuracy:          {test_accuracy:.3f}\n\
Precision:         {test_precision:.3f}\n\
Recall:            {test_recall:.3f}\n\
F1 score:          {test_f1_score:.3f}\n\
CSI score:         {test_csi_score:.3f}')
    
    # Append results
    results.append((
        batch_size, learning_rate, init_hid_dim, val_loss, test_loss, test_accuracy, 
        test_precision, test_recall, test_f1_score, test_csi_score
    ))
    print(f"Finished: Batch Size={batch_size}, Learning Rate={learning_rate}, "
          f"Init Hid Dim={init_hid_dim}, Val Loss={val_loss}, Test Loss={test_loss}")

# Save results to a CSV file
df_results = pd.DataFrame(results, columns=[
    'Batch Size', 'Learning Rate', 'Init Hid Dim', 
    'Validation Loss', 'Test Loss', 'Test Accuracy', 
    'Test Precision', 'Test Recall', 'Test F1 Score', 'Test CSI Score'
])
df_results.to_csv("results_machine_1.csv", index=False)
print("Results saved.")


Testing: Batch size=16, Learning rate=0.05, Init Hid Dim=16
Number of parameters: 1.95e+06.
Epoch: 1 | Training Loss: 0.2352, Validation Loss: 0.2369, Best Validation Loss: 0.2369
Metrics | Accuracy: 0.910, Precision: 0.620, Recall: 0.368, F1-score: 0.462, CSI-score: 0.300
Learning Rate: 0.050000
Metrics saved at: model/losses_metrics/losses_metrics_NDVI_bs16_lr0.05_hid16_epoch1.csv
Average metrics for test dataset using model with best validation loss:

BCE loss:          2.411e-01
Accuracy:          0.910
Precision:         0.697
Recall:            0.385
F1 score:          0.496
CSI score:         0.330
Finished: Batch Size=16, Learning Rate=0.05, Init Hid Dim=16, Val Loss=0.23689965903759003, Test Loss=0.2411319613456726
Testing: Batch size=16, Learning rate=0.05, Init Hid Dim=32
Number of parameters: 7.77e+06.
Epoch: 1 | Training Loss: 0.2178, Validation Loss: 25.7143, Best Validation Loss: 25.7143
Metrics | Accuracy: 0.641, Precision: 0.224, Recall: 0.989, F1-score: 0.365, CSI-sco

OutOfMemoryError: CUDA out of memory. Tried to allocate 1.91 GiB. GPU 0 has a total capacty of 44.34 GiB of which 803.50 MiB is free. Process 2652186 has 5.95 GiB memory in use. Process 90521 has 37.59 GiB memory in use. Of the allocated memory 35.66 GiB is allocated by PyTorch, and 1.61 GiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF